In [2]:
import psycopg2 as p
import getpass
import time, datetime
from datetime import datetime

In [ ]:
start_time = time.time()

In [8]:
password = '*******'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

# MCMIS candidate output

In [9]:
#Candidate output

#12/31/2019 update: Melissa data will parse through the first and last name. Also, data will be output to SharePoint rather than S3 browser. 
#This will be output after running the information through Melissa data
#Will need a place to store the melissa data information


#1/8/2020 update: Primary parser for addresses has been built
#1/9/2020 update: Primary parser for names has been bulit

# dd/mm/YY H:M:S
dt_string = datetime.now().strftime("%Y_%m_%d %H_%M_%S")


query = """
select 

    '' as contact_rep1_status, 
    '' as contact_rep2_status, 
    '' as fs_sic_code,
    sub_q.product_code as fs_product_code,
    '' as company_type_code,
    sub_q.chngdate as mcmis_chngdate,
    sub_q.mcs150_date as mcmis_mcs150_date,
    sub_q.reject_reason as mcmis_update_reason,
    sub_q.class as mcmis_class,
    sub_q.classdef as mcmis_classdef,
    sub_q.census_num as mcmis_census_num,
    initcap(sub_q.name) as mcmis_company_name,
    initcap(sub_q.name_dba) as mcmis_company_name_dba,

    initcap(split_part(sub_q.company_rep1a, ' ', 1)) as mcmis_company_rep1_first_name,

    case
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a ilike '%III%' or sub_q.company_rep1a ilike '%III')) then initcap(split_part(sub_q.company_rep1a, ' ', 2))
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a not ilike '%jr' and sub_q.company_rep1a not ilike '%jr.') and (sub_q.company_rep1a not ilike '%sr' and sub_q.company_rep1a not ilike '%sr.')) then initcap(concat(split_part(sub_q.company_rep1a, ' ', 2), ' ',  split_part(sub_q.company_rep1a, ' ', 3)))
        when (sub_q.count_of_spaces = 3) then initcap(split_part(sub_q.company_rep1a, ' ', 2))
        when (sub_q.count_of_spaces = 2 and sub_q.company_rep1a ilike '%III%') then null
        when (sub_q.count_of_spaces = 2 and (sub_q.company_rep1a not ilike '%jr' and sub_q.company_rep1a not ilike '%jr.') and (sub_q.company_rep1a not ilike '%sr' and sub_q.company_rep1a not ilike '%sr.')) then initcap(split_part(sub_q.company_rep1a, ' ', 2))
    end as mcmis_company_rep1a_middle_name,

    case 
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a ilike '%III%' or sub_q.company_rep1a ilike '%III')) then initcap(split_part(sub_q.company_rep1a, ' ', 3))
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a not ilike '%jr' and sub_q.company_rep1a not ilike '%jr.') and (sub_q.company_rep1a not ilike '%sr' and sub_q.company_rep1a not ilike '%sr.')) then initcap(split_part(sub_q.company_rep1a, ' ', 4))
        when (sub_q.count_of_spaces = 1) then initcap(split_part(sub_q.company_rep1a, ' ', 2)) 
        when (sub_q.count_of_spaces = 2 and company_rep1a ilike '%III%') then initcap(split_part(sub_q.company_rep1a, ' ', 2))
        when (sub_q.count_of_spaces = 2 and (sub_q.company_rep1a ilike '%jr%' or sub_q.company_rep1a ilike '%sr%')) then initcap(split_part(sub_q.company_rep1a, ' ', 2))
        when (sub_q.count_of_spaces = 2 and (sub_q.company_rep1a not ilike '%jr%' or sub_q.company_rep1a not ilike '%sr%')) then initcap(split_part(sub_q.company_rep1a, ' ', 3))
        when (sub_q.count_of_spaces = 3) then initcap(split_part(sub_q.company_rep1a, ' ', 3))
    end as mcmis_company_rep1a_last_name,

    case
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a ilike '%III%' or sub_q.company_rep1a ilike '%III')) then split_part(sub_q.company_rep1a, ' ', 4)
        when (sub_q.count_of_spaces = 3 and (sub_q.company_rep1a not ilike '%jr' and sub_q.company_rep1a not ilike '%jr.') and (sub_q.company_rep1a not ilike '%sr' and sub_q.company_rep1a not ilike '%sr.')) then null
        when (sub_q.count_of_spaces = 3) then split_part(sub_q.company_rep1a, ' ', 4)
        when (sub_q.count_of_spaces = 2 and sub_q.company_rep1a ilike '%jr%') then split_part(sub_q.company_rep1a, ' ', 3)
        when (sub_q.count_of_spaces = 2 and sub_q.company_rep1a ilike '%sr%') then split_part(sub_q.company_rep1a, ' ', 3)
        when (sub_q.count_of_spaces = 2 and sub_q.company_rep1a ilike '%III%') then split_part(sub_q.company_rep1a, ' ', 3)
    end as mcmis_company_rep1_suffix,

    sub_q.company_rep1a,
    '' as fs_original_job_title,
    sub_q.company_rep2 as mcmis_company_rep2,

    initcap(split_part(sub_q.company_rep2a, ' ', 1)) as mcmis_company_rep2_first_name,

    case
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a ilike '%III%' or sub_q.company_rep2a ilike '%III')) then initcap(split_part(sub_q.company_rep2a, ' ', 2))
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a not ilike '%jr' and sub_q.company_rep2a not ilike '%jr.') and (sub_q.company_rep2a not ilike '%sr' and sub_q.company_rep2a not ilike '%sr.')) then initcap(concat(split_part(sub_q.company_rep2a, ' ', 2), ' ',  split_part(sub_q.company_rep2a, ' ', 3)))
        when (sub_q.count_of_spaces2 = 3) then initcap(split_part(sub_q.company_rep2a, ' ', 2))
        when (sub_q.count_of_spaces2 = 2 and sub_q.company_rep2a ilike '%III%') then null
        when (sub_q.count_of_spaces2 = 2 and (sub_q.company_rep2a not ilike '%jr' and sub_q.company_rep2a not ilike '%jr.') and (sub_q.company_rep2a not ilike '%sr' and sub_q.company_rep2a not ilike '%sr.')) then initcap(split_part(sub_q.company_rep2a, ' ', 2))
    end as mcmis_company_rep2_middle_name,

    case 
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a ilike '%III%' or sub_q.company_rep2a ilike '%III')) then initcap(split_part(sub_q.company_rep2a, ' ', 3))
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a not ilike '%jr' and sub_q.company_rep2a not ilike '%jr.') and (sub_q.company_rep2a not ilike '%sr' and sub_q.company_rep2a not ilike '%sr.')) then initcap(split_part(sub_q.company_rep2a, ' ', 4))
        when (sub_q.count_of_spaces2 = 1) then initcap(split_part(sub_q.company_rep2a, ' ', 2)) 
        when (sub_q.count_of_spaces2 = 2 and company_rep2a ilike '%III%') then initcap(split_part(sub_q.company_rep2a, ' ', 2))
        when (sub_q.count_of_spaces2 = 2 and (sub_q.company_rep2a ilike '%jr%' or sub_q.company_rep2a ilike '%sr%')) then initcap(split_part(sub_q.company_rep2a, ' ', 2))
        when (sub_q.count_of_spaces2 = 2 and (sub_q.company_rep2a not ilike '%jr%' or sub_q.company_rep2a not ilike '%sr%')) then initcap(split_part(sub_q.company_rep2a, ' ', 3))
        when (sub_q.count_of_spaces2 = 3) then initcap(split_part(sub_q.company_rep2a, ' ', 3))
    end as mcmis_company_rep2_last_name,

    case
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a ilike '%III%' or sub_q.company_rep2a ilike '%III')) then split_part(sub_q.company_rep2a, ' ', 4)
        when (sub_q.count_of_spaces2 = 3 and (sub_q.company_rep2a not ilike '%jr' and sub_q.company_rep2a not ilike '%jr.') and (sub_q.company_rep2a not ilike '%sr' and sub_q.company_rep2a not ilike '%sr.')) then null
        when (sub_q.count_of_spaces2 = 3) then split_part(sub_q.company_rep2a, ' ', 4)
        when (sub_q.count_of_spaces2 = 2 and sub_q.company_rep2a ilike '%jr%') then split_part(sub_q.company_rep2a, ' ', 3)
        when (sub_q.count_of_spaces2 = 2 and sub_q.company_rep2a ilike '%sr%') then split_part(sub_q.company_rep2a, ' ', 3)
        when (sub_q.count_of_spaces2 = 2 and sub_q.company_rep2a ilike '%III%') then split_part(sub_q.company_rep2a, ' ', 3)
    end as mcmis_company_rep2_suffix,


    sub_q.company_rep2a,
    '' as fs_original_job_title2,
    sub_q.phy_str as mcmis_phy_str,
    regexp_replace(
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                initcap(sub_q.phy_str), 'Avenue', 'Ave'), 
                                                                            ' Suite ', ' Ste '), 
                                                                        E'#.*', ''), 
                                                                    E' Ste.*', ''), 
                                                                E' Apt.*', ''), 
                                                            'Street', 'St'), 
                                                        'Road', 'Rd'), 
                                                    'Drive', 'Dr'), 
                                                'Lane', 'Ln'),
                                            'Circle', 'Cir'),
                                        E'Unit.*', ''),
                                    ' Se ', ' SE '),
                                ' Sw ', ' SW '),
                            ' Nw ', ' NW '),
                        ' Ne ', ' NE '),
                    ' Ne$', ' NE '),
                ' Se$', ' SE '),
            ' Nw$', ' NW '),
        ' Sw$', ' SW ') as mcmis_phy_address1,

    case 
        when (sub_q.phy_str ilike '% ste %' or phy_str ilike '%suite%' or phy_str ~ 'ste\s\d+ ' or phy_str ~ 'ste\#\d*' or phy_str ~ 'ste\s\w*' or phy_str ~ ' ste\d*')then concat('Ste ', regexp_replace(phy_str, '^.* ', '')) 
        end as mcmis_phy_address2,

    case 
        when (sub_q.phy_str ilike '% apt %') then concat('Apt ', regexp_replace(sub_q.phy_str, '^.* ', ''))
        when (sub_q.phy_str ilike '% unit %' or sub_q.phy_str ~ ' unit\d*') then concat('Unit ', regexp_replace(sub_q.phy_str, '^.* ', ''))
        when (sub_q.phy_str ilike '%#%') then concat('#',split_part(sub_q.phy_str, '#', 2)) 
        end as mcmis_phy_address3,

    initcap(sub_q.phy_city) as mcmis_phy_city,
    sub_q.phy_st as mcmis_phy_state_code,
    sub_q.phy_zip as mcmis_phy_zip_code,
    sub_q.phy_natn as mcmis_phy_natn,
    '' as mcmis_phy_county,
    sub_q.undeliv_phy as mcmis_undeliv_phy,
    sub_q.mai_str as mcmis_mai_str,

    -- Additional query will go here for address
        regexp_replace(
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                initcap(sub_q.mai_str), 'Avenue', 'Ave'), 
                                                                            ' Suite ', ' Ste '), 
                                                                        E'#.*', ''), 
                                                                    E' Ste.*', ''), 
                                                                E' Apt.*', ''), 
                                                            'Street', 'St'), 
                                                        'Road', 'Rd'), 
                                                    'Drive', 'Dr'), 
                                                'Lane', 'Ln'),
                                            'Circle', 'Cir'),
                                        E'Unit.*', ''),
                                    ' Se ', ' SE '),
                                ' Sw ', ' SW '),
                            ' Nw ', ' NW '),
                        ' Ne ', ' NE '),
                    ' Ne$', ' NE '),
                ' Se$', ' SE '),
            ' Nw$', ' NW '),
        ' Sw$', ' SW ') as mcmis_mai_address1,

    case 
        when (sub_q.mai_str ilike '% ste %' or sub_q.mai_str ilike '%suite%' or sub_q.mai_str ~ 'ste\s\d+ ' or sub_q.mai_str ~ 'ste\#\d*' or sub_q.mai_str ~ 'ste\s\w*' or sub_q.mai_str ~ ' ste\d*')then concat('Ste ', regexp_replace(sub_q.mai_str, '^.* ', '')) 
        end as mcmis_mai_address2,

    case 
        when (sub_q.mai_str ilike '% apt %') then concat('Apt ', regexp_replace(sub_q.mai_str, '^.* ', ''))
        when (sub_q.mai_str ilike '% unit %' or sub_q.mai_str ~ ' unit\d*') then concat('Unit ', regexp_replace(sub_q.mai_str, '^.* ', ''))
        when (sub_q.mai_str ilike '%#%') then concat('#',split_part(sub_q.mai_str, '#', 2)) 
        end as mcmis_mai_address3,

    initcap(sub_q.mai_city) as mcmis_mai_city,
    sub_q.mai_st as mcmis_mai_state_code,
    sub_q.mai_zip as mcmis_mai_zip,
    sub_q.mai_natn as mcmis_mai_natn,
    '' as mcmis_mai_county,
    sub_q.undeliv_mai as mcmis_undeliv_mai,
    '' as mcmis_mai_latitude,
    '' as mcmis_mai_longitude,
    sub_q.tel_num as mcmis_phone,
    sub_q.fax_num as mcmis_fax,
    '' as fs_website,
    sub_q.email_address as mcmis_email_address,
    sub_q.icc1 as mcmis_icc1,
    sub_q.owntruck as mcmis_owntruck,
    sub_q.trmtruck as mcmis_trmtruck,
    sub_q.trptruck as mcmis_trptruck,
    '' as fs_haz_trucks_owned,
    '' as fs_haz_trucks_leased,
    sub_q.owntract as mcmis_owntract,
    sub_q.trmtract as mcmis_trmtract,
    sub_q.trptract as mcmis_trptract,
    sub_q.owntrail as mcmis_owntrail,
    sub_q.trmtrail as mcmis_trmtrail,
    sub_q.trptrail as mcmis_trptrail,
    '' as fs_haz_trailers_owned,
    '' as fs_haz_trailers_leased,
    sub_q.tot_trucks as mcmis_tot_trucks,
    sub_q.tot_buses as mcmis_tot_buses,
    sub_q.tot_pwr as mcmis_tot_pwr,
    '' as fs_gvw_class_8,
    '' as fs_gvw_class_7,
    '' as fs_gvw_class_6,
    '' as fs_gvw_class_345,
    '' as fs_gvw_class_12,
    sub_q.genfreight as mcmis_genfreight,
    sub_q.household as mcmis_household,
    sub_q.usmail as mcmis_usmail,
    sub_q.intermodal as mcmis_intermodal,
    sub_q.paperprod as mcmis_paperprod,
    sub_q.beverages as mcmis_beverages,
    sub_q.coldfood as mcmis_coldfood,
    sub_q.produce as mcmis_produce,
    sub_q.meat as mcmis_meat,
    sub_q.livestock as mcmis_livestock,
    sub_q.grainfeed as mcmis_grainfeed,
    sub_q.farmsupp as mcmis_farmsupp,
    sub_q.drybulk as mcmis_drybulk,
    sub_q.bldgmat as mcmis_bldgmat,
    sub_q.metalsheet as mcmis_metalsheet,
    sub_q.logpole as mcmis_logpole,
    sub_q.construct as mcmis_construct,
    sub_q.machlrg as mcmis_machlrg,
    sub_q.oilfield as mcmis_oilfield,
    sub_q.waterwell as mcmis_waterwell,
    sub_q.liqgas as mcmis_liqgas,
    sub_q.chem as mcmis_chem,
    sub_q.coalcoke as mcmis_coalcoke,
    sub_q.garbage as mcmis_garbage,
    sub_q.drivetow as mcmis_drivetow,
    sub_q.motorveh as mcmis_motorveh,
    sub_q.mobilehome as mcmis_mobilehome,
    sub_q.utility as mcmis_utility,
    sub_q.cargoothr as mcmis_cargoothr,
    sub_q.othercargo as mcmis_othercargo,
    sub_q.hm_ind as mcmis_hm_ind,
    sub_q.passengers as mcmis_passengers,
    sub_q.carship as mcmis_carship,
    sub_q.crrinter as mcmis_crrinter,
    sub_q.crrhmintra as mcmis_crrhmintra,
    sub_q.crrintra as mcmis_crrintra,
    sub_q.shpinter as mcmis_shpinter,
    sub_q.shpintra as mcmis_shpintra,
    '' as Trailer_type,
    sub_q.mlg150 as mcmis_mlg_150,
    '' as mcmis_avg_mile,
    sub_q.tot_drs as mcmis_tot_drs,
    sub_q.cdl_drs as mcmis_cdl_drs,
    sub_q.mcs150mileageyear as mcmis_mcs150mileageyear,
    sub_q.insertion_flag

FROM (
        SELECT

            a.company_rep1_remove_space as company_rep1a,
            a.company_rep2_remove_space as company_rep2a,
            length(a.company_rep1_remove_space) - length(replace(a.company_rep1_remove_space,' ', '')) AS count_of_spaces,
            length(a.company_rep2_remove_space) - length(replace(a.company_rep2_remove_space, ' ', '')) as count_of_spaces2,
            a.*
        FROM (
                select 
                    m.*,
                    replace(m.company_rep1, '  ', ' ') as company_rep1_remove_space,
                    replace(m.company_rep2, '  ', ' ') as company_rep2_remove_space
                from mcmiscandidate as m
        ) as a

) as sub_q
"""

outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)


with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\Candidates\MCMIS_candidates\candidates' + dt_string + '.csv', 'w') as f:
    cur.copy_expert(outputquery, f)
    


# Fleet table

In [5]:
#Inserting into the fleet table once the insertion_flag has been changed to 'Y'

cur.execute("""

insert into fleetseek.fleet(usdot, company_name, trucks_owned, tractors_owned, trailers_owned, trucks_leased,  
                            tractors_leased, trailers_leased, total_trucks, total_tractors, total_trailers, total_vehicles, total_vehicles_owned, 
                            total_vehicles_leased, dot_hmind, year_mileage, vehicle_miles, drivers, cdl_drivers, product_code, fleet_type_id)
                            
select 
    census_num,  name, owntruck, owntract, owntrail, sum(trptruck+trmtruck), sum(trptract+trmtract), sum(trptrail+trmtrail), 
    sum(owntruck+trptruck+trmtruck), sum(owntract+trmtract+trptract), sum(owntrail+trmtrail+trptrail), sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract),
    sum(owntruck+owntract), sum(trmtruck+trptruck+trmtract+trptract), hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, product_code, fleet_type_id
from mcmiscandidate
where insertion_flag = 'Y'
group by 
    census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, product_code, fleet_type_id;

""")
conn.commit()

In [6]:
#Name changes for new fleets
cur.execute("""
update fleetseek.fleet
set company_name = initcap(company_name)
where company_sort_key is null;


update fleetseek.fleet
set company_name = replace(company_name, 'Llc', 'LLC')
where company_sort_key is null;


update fleetseek.fleet 
set company_sort_key = upper(replace(company_name, ' ', ''))
where company_sort_key is null;
""")
conn.commit()

In [7]:
#Changing the safety rating 
cur.execute("""
update fleetseek.fleet as fs
set safety_rating_type_id = '107'
from (select mc.* from mcmiscandidate as mc, fleetseek.fleet where mc.census_num = fleetseek.fleet.usdot and mc.rating = 'C') as sub_q
where fs.usdot = sub_q.census_num;

update fleetseek.fleet as fs
set safety_rating_type_id = '109'
from (select mc.* from mcmiscandidate as mc, fleetseek.fleet where mc.census_num = fleetseek.fleet.usdot and mc.rating = 'S') as sub_q
where fs.usdot = sub_q.census_num;

update fleetseek.fleet as fs
set safety_rating_type_id = '110'
from (select mc.* from mcmiscandidate as mc, fleetseek.fleet where mc.census_num = fleetseek.fleet.usdot and mc.rating = 'U') as sub_q
where fs.usdot = sub_q.census_num;
""")
conn.commit()

# These next 4 tables all need to have mcmis_id to link back to specific companies in mcmis and then to the respective fleet_id column in fleet

## Location table

In [8]:
#Things to keep in mind: addresses (ste, unit numbers, etc., have to parse through them), update the address3 line after inserting into, making sure that the contact id is pulling the correct location ids
#End goal is to make sure that the correct contact is linked to the correct location_ids
#Join the tables twice so that all the information is included in one row


#What will come from Melissa data: county_fips, latitude, longitude, county_name


#Loop back to 
cur.execute("""

insert into fleetseek.location (address1, city, state_code, zip_code, country_code, active, mcmis_id)

select initcap(phy_str), initcap(phy_city), phy_st, phy_zip, phy_natn, True, mcmis_id
from mcmiscandidate
where insertion_flag = 'Y'

union all 


select initcap(mai_str), initcap(mai_city), mai_st, mai_zip, mai_natn, True, mcmis_id
from mcmiscandidate
where insertion_flag = 'Y'
""")
conn.commit()



## Contact table

In [9]:
#Joining location table to itself and mcmiscandidate to get information for contact 
#Loop back to first_name and last_name to perform regex

cur.execute("""
insert into fleetseek.contact(first_name, last_name, physical_location_id, mailing_location_id, mcmis_id)

select sub_q.company_rep1, sub_q.company_rep1, sub_q.location_id, sub_q.b_location_id, sub_q.m_mcmis_id

from (select 
    a.*, b.location_id as b_location_id, b.address1 as b_address1, b.address2 as b_address2, b.address3 as b_address3, 
    b.city as b_city, b.county_fips as b_county_fips, b.state_code as b_state_code, b.zip_code as b_zip_code, 
    b.country_code as b_country_code, b.latitude as b_latitude, b.longitude as b_longitude, b.county_name as b_county_name, 
    b.mcmis_id as b_mcmis_id, m.company_rep1, m.company_rep2, m.mcmis_id as m_mcmis_id, m.insertion_flag

from fleetseek.location as a
join fleetseek.location as b
on a.mcmis_id = b.mcmis_id
join mcmiscandidate as m
on m.mcmis_id = a.mcmis_id
where a.location_id <> b.location_id and b.location_id > a.location_id and m.insertion_flag = 'Y') as sub_q
""")
conn.commit()

## Email table

In [10]:
cur.execute("""
insert into fleetseek.email (email_address, mcmis_id)
select email_address, mcmis_id
from mcmiscandidate
where insertion_flag = 'Y'
""")
conn.commit()

## Phone Number

In [11]:
cur.execute("""
insert into fleetseek.phone_number (phone_number, mcmis_id, phone_type_id)

select 
    tel_num, mcmis_id, 
    case when (tel_num like '800%' or tel_num like '888%' or tel_num like '866%') then 2
    else 1 end as phone_type_id
    
from mcmiscandidate 
where insertion_flag = 'Y'


union all


select fax_num, mcmis_id, 3
from mcmiscandidate
where insertion_flag = 'Y' and fax_num is not null
""")
conn.commit()

# Fleet_primary and fleet_primary_temp insertion

### This query is by far the most complicated and will likely require the most QA on due to the amount of subqueries and joins that take place

In [ ]:
# 12/11/2019 update:
#This query inserts into both fleet_primary and fleet_primary_temp. 
#The temp table is the one that will be used to insert new data into all the fleet/contact tables (contact_phone, contact_location, fleet_contact, fleet_location)
#Fleet_primary_temp will then be truncated after every new insertion into the respective tables

cur.execute("""
with inserted as --these will be the referenced rows for fleet_primary_temp
    (
    insert into fleetseek.fleet_primary (fleet_id, primary_contact_id, primary_physical_location_id, primary_mailing_location_id, 
                                     company_sort_key, total_vehicles, total_trailers, fleet_type, city, state_code, zip_code, 
                                     country_code, primary_email_id, primary_phone_id, primary_fax_id, primary_toll_free_id)

    select 
        sub_q.fleet_id, 
        sub_q.contact_id, 
        sub_q.physical_location_id, 
        sub_q.mailing_location_id, 
        sub_q.company_sort_key, 
        sub_q.total_vehicles,
        sub_q.total_trailers, 
        sub_q.fleet_type, 
        sub_q.city, 
        sub_q.state_code, 
        sub_q.zip_code, 
        sub_q.country_code, 
        sub_q.email_id, 
        sub_q.phone_number_id,
        case when (sub_q.b_phone_type_id = 3) then sub_q.b_phone_number_id end as primary_fax_id, 
        case when (sub_q.b_phone_type_id = 2) then sub_q.b_phone_number_id end as primary_toll_free_id

    from 
        (select 
            fs.fleet_id, 
            c.contact_id, 
            location_info.location_id as physical_location_id, 
            location_info.b_location_id as mailing_location_id, 
            fs.company_sort_key, 
            fs.total_vehicles, 
            fs.total_trailers, 
            m.reject_reason as fleet_type, 
            initcap(m.phy_city) as city,
            m.phy_st as state_code,
            m.phy_zip as zip_code,
            m.phy_natn as country_code, 
            e.email_id,
            phone_info.phone_number_id,
            phone_info.phone_type_id,
            phone_info.phone_number,
            phone_info.b_phone_number_id,
            phone_info.b_phone_type_id,
            phone_info.b_phone_number

        from fleetseek.fleet as fs
        join mcmiscandidate as m
        on fs.usdot = m.census_num
        join (  
                select 
                    a.mcmis_id, 
                    a.location_id, 
                    b.location_id as b_location_id, 
                    b.mcmis_id as b_mcmis_id

                from fleetseek.location as a
                join fleetseek.location as b
                on a.mcmis_id = b.mcmis_id
                where a.location_id <> b.location_id and b.location_id > a.location_id
             ) as location_info
        on m.mcmis_id = location_info.mcmis_id
        join fleetseek.email as e
        on location_info.mcmis_id = e.mcmis_id
        join fleetseek.contact as c
        on e.mcmis_id = c.mcmis_id
        join 
               (
                select 
                    a.phone_number_id, 
                    a.phone_type_id, 
                    a.phone_number, 
                    nullif(b.phone_number_id, a.phone_number_id) as b_phone_number_id, 
                    nullif(b.phone_type_id, a.phone_type_id) as b_phone_type_id, 
                    nullif(b.phone_number, a.phone_number) as b_phone_number, 
                    b.mcmis_id
                    
                from fleetseek.phone_number as a
                join fleetseek.phone_number as b
                on a.mcmis_id = b.mcmis_id
                join (
                    select 
                        mcmis_id, 
                        count(mcmis_id) as count_of_id
                        
                    from fleetseek.phone_number
                    where mcmis_id is not null
                    group by mcmis_id
                ) as c
                on b.mcmis_id = c.mcmis_id
                where a.phone_type_id = 1 and c.count_of_id = 1


                union all

                select 
                    a.phone_number_id, 
                    a.phone_type_id, 
                    a.phone_number, 
                    b.phone_number_id as b_phone_number_id, 
                    b.phone_type_id as b_phone_type_id, 
                    b.phone_number as b_phone_number, 
                    b.mcmis_id
                    
                from fleetseek.phone_number as a
                join fleetseek.phone_number as b
                on a.mcmis_id = b.mcmis_id
                where a.phone_type_id = 1 and a.phone_number_id <> b.phone_number_id
                
                )
                
         as phone_info
         on c.mcmis_id = phone_info.mcmis_id
         where m.insertion_flag = 'Y'
        ) as sub_q
        returning 
            fleet_id, 
            primary_contact_id, 
            primary_physical_location_id, 
            primary_mailing_location_id, 
            company_sort_key, total_vehicles,
            total_trailers, 
            UPPER(fleet_type) as fleet_type, 
            city, 
            state_code, 
            zip_code, 
            country_code, 
            primary_email_id, 
            primary_phone_id, 
            primary_fax_id,
            primary_toll_free_id
        )
insert into fleetseek.fleet_primary_temp (fleet_id, primary_contact_id, primary_physical_location_id, primary_mailing_location_id, company_sort_key, total_vehicles,
                                         total_trailers, fleet_type, city, state_code, zip_code, country_code, primary_email_id, primary_phone_id,
                                         primary_fax_id, primary_toll_free_id)
                                         
select 
    fleet_id, 
    primary_contact_id, 
    primary_physical_location_id, 
    primary_mailing_location_id, 
    company_sort_key, 
    total_vehicles,
    total_trailers, 
    fleet_type, city, 
    state_code, 
    zip_code, 
    country_code, 
    primary_email_id, 
    primary_phone_id, 
    primary_fax_id, 
    primary_toll_free_id
    
from inserted 
""")
conn.commit()

## Contact_email

In [19]:
cur.execute("""
insert into fleetseek.contact_email (contact_id, email_id)
select primary_contact_id, primary_email_id
from fleetseek.fleet_primary_temp
""")
conn.commit()

## Contact_phone

In [22]:
cur.execute("""
insert into fleetseek.contact_phone(contact_id, phone_number_id)
select primary_contact_id, primary_phone_id
from fleetseek.fleet_primary_temp

union all

select primary_contact_id, primary_fax_id
from fleetseek.fleet_primary_temp
where primary_fax_id is not null

union all

select primary_contact_id, primary_toll_free_id
from fleetseek.fleet_primary_temp
where primary_toll_free_id is not null
order by primary_contact_id
""")
conn.commit()

## Fleet_contact

In [29]:
cur.execute("""
insert into fleetseek.fleet_contact (fleet_id, contact_id)
select fleet_id, primary_contact_id
from fleetseek.fleet_primary_temp
""")
conn.commit()

## Fleet_location

In [30]:
#May need to be union or union all (come back after more candidates are present)
cur.execute("""
insert into fleetseek.fleet_location (fleet_id, location_id, location_type)
select fleet_id, primary_physical_location_id, case when primary_physical_location_id < primary_mailing_location_id then 'physical' end as location_type
from fleetseek.fleet_primary_temp

union all 

select fleet_id, primary_mailing_location_id, case when primary_mailing_location_id > primary_physical_location_id then 'mailing' end as location_type
from fleetseek.fleet_primary_temp
""")
conn.commit()

## Fleet_commodity

In [42]:
cur.execute("""
insert into fleetseek.fleet_commodity(fleet_id, commodity_id, sort_order)
select sub_q.fleet_id, sub_q.c_commodity_id, sub_q.sort_order
from (

    select fp.fleet_id, fp.commodity_id as fp_commodity_id, c.commodity_id as c_commodity_id, c.sort_order
    from fleetseek.fleet_primary_temp as fp
    join fleetseek.commodity as c 
    on c.commodity_id = any(fp.commodity_id)
) as sub_q
""")
conn.commit()

## Fleet_phone

In [36]:
#May or may not need to maintain this table?
cur.execute("""
insert into fleetseek.fleet_phone (fleet_id, phone_number_id)

select fleet_id, primary_phone_id
from fleetseek.fleet_primary_temp

union all

select fleet_id, primary_fax_id
from fleetseek.fleet_primary_temp
where primary_fax_id is not null

union all

select fleet_id, primary_toll_free_id
from fleetseek.fleet_primary_temp
where primary_toll_free_id is not null
order by fleet_id
""")
conn.commit()

## Truncate fleet_primary_temp

In [40]:
cur.execute("""
truncate fleetseek.fleet_primary_temp
""")
conn.commit()

In [ ]:
end_time = time.time() - start_time
print(end_time)